# Usage

## Initialise Client

On initialisation, the client will post the
conformance classes which are list of available
addon functions available for a certain STAC
endpoint.

First setup of the client, use a STAC endpoint URL
as client parameter:

In [ ]:
from client.client import StacPyClient
import json

url = 'https://api.stac.ceda.ac.uk'
Client = StacPyClient(url=url)

In [ ]:
# Display function
def display(obj: dict):
    print(json.dumps(obj, indent=4))

## Basic GET usages

### GET Collections

**/collections**

Retrieve a list of collections available at from the STAC endpoint.

In [ ]:
collections = Client.get_collections()
# returns list of collections JSON

display(collections)

### GET Collection

**/collections/{collectionId}**

Retreive a collection, given an collection ID, from the STAC endpoint.

In [ ]:
collection_id = collections[0]['id']
collection = Client.get_collection(collection_id=collection_id)

# returns a collection object
display(collection)


### GET Itemcollection

**/collections/{collectionId}/items**

Retrieve a list of items of a specific collection.

In [ ]:
items = Client.get_itemcollection(collection_id=collection_id)
# Returns an item collection object which is a list of items from a collection id,
# or get an item collection from a collection object:
items = collection.get_items()

display(items)

### GET Item

**/collections/{collectionId}/items/{itemId}**

Retrieve an item object, from a specific collection, given an item ID.

In [ ]:
item_id = items['features'][0]['id']
item = Client.get_item(collection_id=collection_id, item_id=item_id)
# Returns an item object,
# or get an item from a collection object given an item ID
item = collection.get_items(item_id=item_id)

display(item)

### GET Assets

Retrieve a list of assets of a given item.

In [ ]:
assets = item.assets
# Returns a list of assets JSON of a specific item.

display(assets)

### Queryables

**/queryables**

Retrieve a JSON response for properties to query search.

In [ ]:
queryables = Client.queryables()

display(queryables)

### Collection Queryables

**/collections/{collectionId}/queryables**

Retrieve a JSON response for properties to query in a collection.

In [ ]:
queryables = Client.collection_queryables(collection_id=collection_id)

display(queryables)

## Search

Usages examples of how search using the python wrapper client.
(See Conformance classes `item-search` for capabilities)

### Basic Usage:

Search the STAC endpoint by filtering through these optional keys:
* collections: list of collection IDs
* ids: list of item IDs
* bbox: list of integers for bounding box
* datetime: string of open/closed ended dates or single date.
* limit: number of items to list in one page. *Default 10*.

In [ ]:
results = Client.search()
# returns every item available

results = Client.search(
    collections=['Fj3reHsBhuk7QqVbt7P-'],
    ids=['2ef41eee0710db0a04c7089b3da3ee6b'],
    datetime='2018-02-12T00:00:00Z/2018-03-18T12:31:12Z',
    limit=10
)
# returns an item collection object of any item that satisfies these arguments.
# Note: this specific search query won't match anything, though mix and match
# the parameters with different values and see what comes up. All are optional.

display(results)

### Free Text Search

Free text search is provided by the client, using a positional argument as the first
or by the `q` parameter.
Free text search supports case-insensitive and partial search.

In [ ]:
# All these queries end up with the same search result:
result = Client.search(q="AerChemMIP")
print(result['context']['matched'])

result = Client.search("AerChemMIP")
print(result['context']['matched'])

result = Client.search("aerchemmip")
print(result['context']['matched'])

# Can use wildcard queries for partial matches
result = Client.search("AerChem*") # the star, *, is a wildcard symbol.


# It is also possible to add other arguments to the free text search:
result = Client.search("aerchem*", datetime="2000-11-01T00:00Z/..")
display(result)

### Facet Search
Filter the search result based on facets of an item. The facet request body
should use a dictionary with valid facets found with queryables, labeled under
the filter argument.

In [ ]:

# The filter is the label for facet search where it consists of a dictionary
# made up of the facet as the key and value what to filter for.
# Just like others, it can be queried with other arguments:
result = Client.search(
    filter={
        "eq": [
            {"property": "variable"},
            "clt"
        ]
    }
)

display(result)